# Partie 1 - Mesures d'Impurete

Ce notebook presente les differentes mesures d'impurete utilisees dans les arbres de decision:
- Indice de Gini
- Entropie de Shannon
- Erreur de classification

**Auteur**: Projet Data Mining - Arbres de Decision

## 1. Importation des bibliotheques

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configuration pour de meilleurs graphiques
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 2. Definitions mathematiques

### 2.1 Indice de Gini
L'indice de Gini mesure la probabilite qu'un element choisi au hasard soit mal classe.

$$Gini(t) = 1 - \sum_{i=1}^{c} p_i^2$$

ou $p_i$ est la proportion d'elements de la classe $i$ dans le noeud $t$.

### 2.2 Entropie de Shannon
L'entropie mesure le desordre ou l'incertitude dans un ensemble.

$$Entropie(t) = -\sum_{i=1}^{c} p_i \log_2(p_i)$$

### 2.3 Erreur de classification
L'erreur de classification est la proportion d'elements qui ne sont pas dans la classe majoritaire.

$$Erreur(t) = 1 - \max_i(p_i)$$

## 3. Implementation des fonctions d'impurete

In [ ]:
def gini(counts):
    """
    Calcule l'indice de Gini pour une distribution donnee.
    
    Parametres:
    -----------
    counts : list
        Liste contenant le nombre d'elements dans chaque classe.
        Exemple: [10, 10] pour 10 positifs et 10 negatifs.
    
    Retourne:
    ---------
    float
        L'indice de Gini (entre 0 et 0.5 pour un probleme binaire).
    """
    total = sum(counts)
    if total == 0:
        return 0
    probs = [c / total for c in counts]
    return 1 - sum(p**2 for p in probs)


def entropy(counts):
    """
    Calcule l'entropie de Shannon pour une distribution donnee.
    
    Parametres:
    -----------
    counts : list
        Liste contenant le nombre d'elements dans chaque classe.
    
    Retourne:
    ---------
    float
        L'entropie (entre 0 et log2(c) ou c est le nombre de classes).
    """
    total = sum(counts)
    if total == 0:
        return 0
    probs = [c / total for c in counts if c > 0]
    return -sum(p * np.log2(p) for p in probs)


def classification_error(counts):
    """
    Calcule l'erreur de classification pour une distribution donnee.
    
    Parametres:
    -----------
    counts : list
        Liste contenant le nombre d'elements dans chaque classe.
    
    Retourne:
    ---------
    float
        L'erreur de classification (entre 0 et 1 - 1/c).
    """
    total = sum(counts)
    if total == 0:
        return 0
    return 1 - max(counts) / total

## 4. Exemples numeriques

Nous allons calculer les trois mesures d'impurete pour differentes distributions:
- **10/10**: Distribution parfaitement equilibree
- **18/2**: Distribution tres desequilibree (presque pure)
- **9/1**: Distribution desequilibree
- **5/5**: Distribution equilibree (petit echantillon)
- **1/9**: Distribution desequilibree (inverse de 9/1)

In [ ]:
# Definition des cas a tester
cases = {
    "10/10 (equilibre)": [10, 10],
    "18/2 (tres pur)": [18, 2],
    "9/1 (desequilibre)": [9, 1],
    "5/5 (equilibre)": [5, 5],
    "1/9 (desequilibre)": [1, 9],
    "20/0 (pur)": [20, 0],
    "15/5": [15, 5]
}

# Calcul des mesures pour chaque cas
data = []
for name, counts in cases.items():
    data.append({
        "Cas": name,
        "Positifs": counts[0],
        "Negatifs": counts[1],
        "Gini": round(gini(counts), 4),
        "Entropie": round(entropy(counts), 4),
        "Erreur": round(classification_error(counts), 4)
    })

# Creation du DataFrame
df_results = pd.DataFrame(data)
print("Tableau comparatif des mesures d'impurete:")
print("=" * 80)
df_results

## 5. Visualisation graphique

In [ ]:
# Graphique en barres comparatif
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(cases))
width = 0.25

bars1 = ax.bar(x - width, df_results['Gini'], width, label='Gini', color='#2ecc71')
bars2 = ax.bar(x, df_results['Entropie'], width, label='Entropie', color='#3498db')
bars3 = ax.bar(x + width, df_results['Erreur'], width, label='Erreur', color='#e74c3c')

ax.set_xlabel('Distribution des classes')
ax.set_ylabel('Valeur de l\'impurete')
ax.set_title('Comparaison des mesures d\'impurete pour differentes distributions')
ax.set_xticks(x)
ax.set_xticklabels([c.split()[0] for c in cases.keys()], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Evolution de l'impurete en fonction de la proportion

In [ ]:
# Courbes continues d'impurete
proportions = np.linspace(0.001, 0.999, 100)

gini_values = []
entropy_values = []
error_values = []

for p in proportions:
    counts = [p * 100, (1-p) * 100]
    gini_values.append(gini(counts))
    entropy_values.append(entropy(counts))
    error_values.append(classification_error(counts))

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(proportions, gini_values, label='Gini', linewidth=2, color='#2ecc71')
ax.plot(proportions, entropy_values, label='Entropie', linewidth=2, color='#3498db')
ax.plot(proportions, error_values, label='Erreur de classification', linewidth=2, color='#e74c3c')

ax.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5, label='p=0.5 (max impurete)')

ax.set_xlabel('Proportion de la classe positive (p)')
ax.set_ylabel('Valeur de l\'impurete')
ax.set_title('Evolution des mesures d\'impurete en fonction de la proportion de classes')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Analyse et interpretation

### Observations:

1. **Noeud pur (20/0 ou 0/20)**:
   - Toutes les mesures valent 0
   - C'est l'objectif a atteindre lors de la construction de l'arbre

2. **Distribution equilibree (10/10 ou 5/5)**:
   - Impurete maximale
   - Gini = 0.5, Entropie = 1.0, Erreur = 0.5

3. **Distribution desequilibree**:
   - Plus la distribution est desequilibree, plus l'impurete diminue
   - L'entropie est plus sensible aux petites variations pres des extremes

### Proprietes des mesures:

| Mesure | Valeur min | Valeur max | Sensibilite |
|--------|------------|------------|-------------|
| Gini   | 0          | 0.5        | Moyenne     |
| Entropie | 0        | 1.0        | Elevee      |
| Erreur | 0          | 0.5        | Faible      |

In [ ]:
print("Fin du notebook - Mesures d'impurete")
print("Les fonctions definies peuvent etre reutilisees dans les notebooks suivants.")